### Packages

In [9]:
# NumCosmo
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

import matplotlib.pyplot as plt
import scipy
import numpy as np
# Necessary functions:

from numpy import sqrt
from numpy import log10
from numpy import sinh
from numpy import log as ln
from scipy.integrate import quad

c = 3*(10**8)

### NumCosmo Settings

In [10]:
Ncm.cfg_init ()
cosmo = Nc.HICosmoDEXcdm() #Cosmology object

### Data

In [30]:
ser = Ncm.Serialize.new(0)
Ncm.cfg_init ()
data = ser.from_file("/home/cinthia/NumCosmo/data/nc_data_snia_diag_legacy.obj")
lenz = data.y.len()

muobs = []
sigma = []
zobs = []
for i in range(lenz):
    mi = data.x.get(i)
    si = data.sigma.get(i)
    zi = data.y.get(i)
    
    muobs.append(mi)
    sigma.append(si)
    zobs.append(zi)

muobs = np.array(muobs)
zobs = np.array(zobs)
sigma = np.array(sigma)

### Likelihood (-2ln L)

In [46]:
def integ(f, a, b):
    i = quad(lambda x:f, a, b)
    return i[0]

def mu(z, theta):
    H0 = theta[0]
    Omega_l0 = theta[1]
    Omega_k0 = theta[2]
    Omega_m0 = theta[3]
    Omega_r0 = theta[4]
   
    E2z = Omega_l0 + Omega_k0*((1 + z) ** 2) + Omega_m0*((1 + z) ** 3) + Omega_r0*((1 + z) ** 4) 
    Ez = sqrt(E2z)
    
    I = integ(1/Ez, 0, z.all()) #Calcula a integral de 1/Ez de 0 a z
    
    dc = (c / H0) * I       
    Dc = (H0 / c) * dc
    
    if Omega_k0 == 0:
        Dt = Dc
    else:
        Dt = sinh((sqrt(Omega_k0)) * Dc) / sqrt(Omega_k0)

    Dl = (1 + z) * Dt

    mu = 5 * log10(Dl) + 25 + 5 * log10((c / H0))
    
    return mu 


def f_2lnL(z, theta, mu_values, sigma_values):
    arg = []
    for i in range(lenz):
        argi = ((mu(z,theta) - mu_values[i]) ** 2) / (sigma_values[i]) ** 2
        arg.append(argi)
    
        return sum(arg)    


In [47]:
def func_parameters(theta):
    f = f_2lnL(zobs,theta,muobs,sigma)
    return f
    
def fit():
    x0 = [70,0.27, 0, 0.73, 0]
    x0_np = np.array(x0)
    fit_parameters = scipy.optimize.minimize(func_parameters, x0)
    return 

f = fit()

print(f)
x0 = [70,0.27, 0, 0.73, 0]
z1 = zobs[0]
m = mu(z1,x0)

print(m)

TypeError: only size-1 arrays can be converted to Python scalars

### Matriz de Fisher